In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [11]:
# Define model
class ModelClass(nn.Module):
    def __init__(self):
        super(ModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = ModelClass()


In [12]:
model.forward

<bound method ModelClass.forward of ModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [13]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.conv1.register_forward_hook(layer1_hook)
model.conv2.register_forward_hook(layer1_hook)

In [14]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [15]:
outputs

[tensor([[[[ 0.7518, -0.5600,  1.0196,  ..., -0.5894,  0.5885,  0.1745],
           [ 0.4496, -0.2529, -0.7520,  ...,  0.2933,  0.3209, -0.2656],
           [ 0.6473, -0.2342,  0.6649,  ..., -0.1355,  0.8620, -0.0911],
           ...,
           [ 0.5096,  0.2246, -0.7652,  ...,  0.8910, -0.7117,  0.3892],
           [ 0.1611,  0.3925,  0.4996,  ...,  0.3552,  1.2251,  0.6712],
           [ 1.0476, -0.4419, -0.1850,  ..., -0.5570,  0.9512, -0.1513]],
 
          [[ 0.1275,  1.2788, -0.7977,  ..., -1.4047,  0.6842, -0.1119],
           [-1.0779,  0.0193, -0.3847,  ...,  1.2812,  0.0425,  0.0020],
           [ 0.7555,  0.3980,  0.0038,  ..., -0.2910,  0.5024,  0.2132],
           ...,
           [-0.1485,  0.2374, -0.2452,  ...,  0.3371,  0.0480,  0.1256],
           [ 1.0519, -0.1595,  1.2014,  ...,  0.3973,  0.2273, -0.2677],
           [-0.7109,  0.3102,  0.3369,  ..., -0.7266, -0.3776, -0.9950]],
 
          [[ 0.4401, -0.4618, -0.4290,  ..., -0.3088, -0.2392, -0.0860],
           [ 

In [ ]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


In [16]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 28, 28])
torch.Size([1, 16, 10, 10])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

References: 
* default：https://github.com/pytorch/pytorch/tree/master/torch/nn/modules
* refer to `reset_parameters` under `function`
* PyTorch Initialization function：https://pytorch.org/docs/stable/nn.init.html

In [21]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.ones_(m.bias)

model.apply(weights_init)

ModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [22]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


